# **Deep Hedging**
# Buchkov Viacheslav

In [ ]:
import abc
import math
import os
import random
import sys
from functools import lru_cache
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm

# You may add any imports you need
from torch.cuda.amp import GradScaler

RANDOM_SEED = 12

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(RANDOM_SEED)

In [ ]:
N_DAYS = 5

## Getting spot data.

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

ROOT_PATH = Path("dataset")
PATH = Path("/content/gdrive/MyDrive/")

Mounted at /content/gdrive


In [ ]:
!unzip -q '/content/gdrive/MyDrive/quotes_USDRUB_resample.zip' -d './'

In [ ]:
!unzip -q '/content/gdrive/MyDrive/quotes_EURRUB_resample.zip' -d './'

In [ ]:
usdrub = pd.read_csv("quotes_USDRUB_resample_1min.csv")
usdrub["timestamp"] = pd.to_datetime(usdrub["timestamp"])
usdrub = usdrub.set_index("timestamp")
usdrub.head()

,Bid,Ask
timestamp,,
2017-01-03 07:00:00+00:00,60.973760,61.016500
2017-01-03 07:01:00+00:00,61.010855,61.068045
2017-01-03 07:02:00+00:00,61.037220,61.069850
2017-01-03 07:03:00+00:00,61.011955,61.048690
2017-01-03 07:04:00+00:00,61.021250,61.052835


In [ ]:
eurrub = pd.read_csv("quotes_EURRUB_resample_1min.csv")
eurrub["timestamp"] = pd.to_datetime(eurrub["timestamp"])
eurrub = eurrub.set_index("timestamp")
eurrub.head()

,Bid,Ask
timestamp,,
2017-01-03 07:00:00+00:00,63.903150,64.343550
2017-01-03 07:01:00+00:00,63.881855,64.289995
2017-01-03 07:02:00+00:00,63.914725,65.362045
2017-01-03 07:03:00+00:00,63.879200,64.284930
2017-01-03 07:04:00+00:00,63.886790,64.039440


In [ ]:
tol = pd.Timedelta('1 min')
spot = pd.merge_asof(left=usdrub, right=eurrub, right_index=True, left_index=True, direction='nearest', tolerance=tol)

In [ ]:
spot.head()

,Bid_x,Ask_x,Bid_y,Ask_y
timestamp,,,,
2017-01-03 07:00:00+00:00,60.973760,61.016500,63.903150,64.343550
2017-01-03 07:01:00+00:00,61.010855,61.068045,63.881855,64.289995
2017-01-03 07:02:00+00:00,61.037220,61.069850,63.914725,65.362045
2017-01-03 07:03:00+00:00,61.011955,61.048690,63.879200,64.284930
2017-01-03 07:04:00+00:00,61.021250,61.052835,63.886790,64.039440


In [ ]:
spot.index.min(), spot.index.max()

(Timestamp('2017-01-03 07:00:00+0000', tz='UTC'),
 Timestamp('2024-04-30 15:51:00+0000', tz='UTC'))

## Getting rates data.

In [ ]:
rub_rates = pd.read_csv("/content/gdrive/MyDrive/dynamic.csv", sep=";")

rub_rates["timestamp"] = pd.to_datetime(rub_rates["tradedate"] + " " + rub_rates["tradetime"], format="%d.%m.%Y %H:%M:%S")
rub_rates = rub_rates.set_index("timestamp")
rub_rates.drop(["tradedate", "tradetime"], axis=1, inplace=True)

rub_rates = rub_rates.apply(lambda row: row.str.replace(",", "."))
rub_rates = rub_rates.astype(float)
rub_rates.head()

,B1,B2,B3,T1,G1,G2,G3,G4,G5,G6,G7,G8,G9
timestamp,,,,,,,,,,,,,
2014-01-06 12:21:16,877.951361,-311.324633,51.105265,4.836731,0.000000,0.000000,-0.235430,-0.602083,-0.725340,-0.341294,0.683989,0.0,0.0
2014-01-08 12:41:22,879.619947,-312.611788,51.560662,4.824178,-0.005661,-0.005661,-0.214698,-1.641290,-4.821431,0.328790,3.391207,0.0,0.0
2014-01-09 18:38:19,876.971884,-320.831620,56.936812,4.448947,0.065004,-3.996982,-4.358512,0.140076,0.524525,4.560520,1.350378,0.0,0.0
2014-01-10 18:38:17,875.118031,-329.716005,61.089853,4.148542,0.018092,-2.428168,-3.204718,0.605912,2.373884,3.420259,0.019879,0.0,0.0
2014-01-13 18:37:26,878.212981,-325.573772,60.784188,4.195013,2.689991,3.672235,-8.583068,-1.823289,-0.645657,1.413068,-0.751966,0.0,0.0


In [ ]:
usd_rates = pd.read_csv("/content/gdrive/MyDrive/feds200628.csv")
usd_rates["timestamp"] = pd.to_datetime(usd_rates["Date"])
usd_rates.drop(["Date"], axis=1, inplace=True)
usd_rates = usd_rates.set_index("timestamp")
usd_rates.head()

,BETA0,BETA1,BETA2,BETA3,SVEN1F01,SVEN1F04,SVEN1F09,SVENF01,SVENF02,SVENF03,...,SVENY23,SVENY24,SVENY25,SVENY26,SVENY27,SVENY28,SVENY29,SVENY30,TAU1,TAU2
timestamp,,,,,,,,,,,,,,,,,,,,,
1961-06-14,3.917606,-1.277955,-1.949397,0.0,3.8067,3.9562,NaN,3.5492,3.8825,3.9149,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.339218,-999.99
1961-06-15,3.978498,-1.257404,-2.247617,0.0,3.8694,4.0183,NaN,3.5997,3.9460,3.9763,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.325775,-999.99
1961-06-16,3.984350,-1.429538,-1.885024,0.0,3.8634,4.0242,NaN,3.5957,3.9448,3.9811,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.348817,-999.99
1961-06-19,4.004379,-0.723311,-3.310743,0.0,3.9196,4.0447,NaN,3.6447,3.9842,4.0035,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.282087,-999.99
1961-06-20,3.985789,-0.900432,-2.844809,0.0,3.8732,4.0257,NaN,3.5845,3.9552,3.9840,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.310316,-999.99


In [ ]:
class YieldCurve:
    TARGET_COLUMN = "ytm"
    DISCOUNT_FACTOR_COLUMN = "discount_factor"
    FWD_RATE_COLUMN = "fwd_rate"

    def __init__(self, initial_terms: np.array, *args, **kwargs) -> None:
        self._rates_df = None
        self._discount_factors = None
        self._instant_fwd_rate = None

        self._initialize(initial_terms)

    def _initialize(self, terms: np.array) -> None:
        self.create_curve(terms=terms)

    @abc.abstractmethod
    def get_rates(self, terms: list[float]) -> np.array:
        pass

    def create_curve(self, terms: list[float]) -> None:
        self._rates_df = pd.DataFrame(self.get_rates(terms), index=terms, columns=[self.TARGET_COLUMN])
        # self._create_discount_factors()
        # self._create_instant_fwd_rates()

    def _create_discount_factors(self) -> pd.DataFrame:
        if self._rates_df is None:
            raise ValueError("Rate data is not fitted yet!")
        discount_factors = np.exp(-self._rates_df[self.TARGET_COLUMN] * self._rates_df.index)
        self._discount_factors = pd.DataFrame(discount_factors, index=self._rates_df.index,
                                              columns=[self.DISCOUNT_FACTOR_COLUMN])
        return self._discount_factors

    def _create_instant_fwd_rates(self) -> pd.DataFrame:
        if self._discount_factors is None:
            raise ValueError("Discount factor data is not fitted yet!")

        t_old = self._rates_df.index[0]
        instant_fwd_rates = []
        for t in self._rates_df.index[1:]:
            dt = t - t_old
            instant_fwd_rates.append(-1 / dt * (np.log(
                self._discount_factors.loc[t, self.DISCOUNT_FACTOR_COLUMN] / self._discount_factors.loc[
                    t_old, self.DISCOUNT_FACTOR_COLUMN])))
            t_old = t
        self._instant_fwd_rate = pd.DataFrame(instant_fwd_rates, index=self._rates_df.index[1:],
                                              columns=[self.FWD_RATE_COLUMN])
        return self._instant_fwd_rate

    @property
    def curve_df(self) -> pd.DataFrame:
        if self._rates_df is None:
            raise ValueError("Rate data is not fitted yet! Call .create_curve() first.")
        return self._rates_df

    @property
    def discount_factors_df(self) -> pd.DataFrame:
        return self._discount_factors

    @property
    def instant_fwd_rates_df(self) -> pd.DataFrame:
        return self._instant_fwd_rate

    @staticmethod
    def _find_point(curve: pd.DataFrame, term: float) -> float:
        index = np.absolute(curve.index - term).argmin()
        return curve.iloc[index].values[0]

    @lru_cache(maxsize=None)
    def get_rate(self, term: float) -> float:
        return self._find_point(self._rates_df, term)

    @lru_cache(maxsize=None)
    def get_discount_factor(self, term: float) -> float:
        return self._find_point(self._discount_factors, term)

    @lru_cache(maxsize=None)
    def get_instant_fwd_rate(self, term: float) -> float:
        return self._find_point(self._instant_fwd_rate, term)

In [ ]:
class NelsonSiegelCurve(YieldCurve):
    def __init__(self, b0: float, b1: float, b2: float, tau: float,
                 initial_terms: np.array = np.linspace(1 / 365, 25., 100)) -> None:
        self.b0 = b0
        self.b1 = b1
        self.b2 = b2
        self.tau = tau

        super().__init__(initial_terms)

    def get_rates(self, terms: list[float]) -> np.array:
        terms = np.array(terms)
        rates = self.b0 + (self.b1 + self.b2) * self.tau / terms * (1 - np.exp(-terms / self.tau)) - self.b2 * np.exp(
            -terms / self.tau)
        return rates / 100

## Merging data.

In [ ]:
tol = pd.Timedelta('24 hours')
df = pd.merge_asof(left=spot, right=rub_rates, right_index=True, left_index=True, direction='nearest', tolerance=tol)
df = pd.merge_asof(left=df, right=usd_rates, right_index=True, left_index=True, direction='nearest', tolerance=tol)
df.head()

,Bid_x,Ask_x,Bid_y,Ask_y,B1,B2,B3,T1,G1,G2,...,SVENY23,SVENY24,SVENY25,SVENY26,SVENY27,SVENY28,SVENY29,SVENY30,TAU1,TAU2
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-03 07:00:00+00:00,60.973760,61.016500,63.903150,64.343550,924.070641,-111.376396,-260.204466,4.411702,-5.858851,8.954092,...,3.0356,3.0709,3.1064,3.142,3.1779,3.2137,3.2496,3.2853,1.776206,11.530964
2017-01-03 07:01:00+00:00,61.010855,61.068045,63.881855,64.289995,924.070641,-111.376396,-260.204466,4.411702,-5.858851,8.954092,...,3.0356,3.0709,3.1064,3.142,3.1779,3.2137,3.2496,3.2853,1.776206,11.530964
2017-01-03 07:02:00+00:00,61.037220,61.069850,63.914725,65.362045,924.070641,-111.376396,-260.204466,4.411702,-5.858851,8.954092,...,3.0356,3.0709,3.1064,3.142,3.1779,3.2137,3.2496,3.2853,1.776206,11.530964
2017-01-03 07:03:00+00:00,61.011955,61.048690,63.879200,64.284930,924.070641,-111.376396,-260.204466,4.411702,-5.858851,8.954092,...,3.0356,3.0709,3.1064,3.142,3.1779,3.2137,3.2496,3.2853,1.776206,11.530964
2017-01-03 07:04:00+00:00,61.021250,61.052835,63.886790,64.039440,924.070641,-111.376396,-260.204466,4.411702,-5.858851,8.954092,...,3.0356,3.0709,3.1064,3.142,3.1779,3.2137,3.2496,3.2853,1.776206,11.530964


In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

df["usd_curve"] = df.progress_apply(lambda row: NelsonSiegelCurve(b0=row["BETA0"], b1=row["BETA1"], b2=row["BETA2"], tau=row["TAU1"]), axis=1)

  0%|          | 0/1488719 [00:00<?, ?it/s]

In [ ]:
tqdm.pandas()

df["rub_curve"] = df.progress_apply(lambda row: NelsonSiegelCurve(b0=row["B1"], b1=row["B2"], b2=row["B3"], tau=row["T1"]), axis=1)

  0%|          | 0/1488719 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
tqdm.pandas()
df["rub_rate"] = df["rub_curve"].progress_apply(lambda curve: curve.get_rate(N_DAYS / 365) / 100)

  0%|          | 0/1488719 [00:00<?, ?it/s]

In [ ]:
df["usd_rate"] = df["usd_curve"].progress_apply(lambda curve: curve.get_rate(N_DAYS / 365))

  0%|          | 0/1488719 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Saving final dataset.

In [ ]:
data = df[["Bid_x", "Ask_x", "Bid_y", "Ask_y", "rub_rate", "usd_rate"]]
data.head()

,Bid_x,Ask_x,Bid_y,Ask_y,rub_rate,usd_rate
timestamp,,,,,,
2017-01-03 07:00:00+00:00,60.973760,61.016500,63.903150,64.343550,0.081265,0.00672
2017-01-03 07:01:00+00:00,61.010855,61.068045,63.881855,64.289995,0.081265,0.00672
2017-01-03 07:02:00+00:00,61.037220,61.069850,63.914725,65.362045,0.081265,0.00672
2017-01-03 07:03:00+00:00,61.011955,61.048690,63.879200,64.284930,0.081265,0.00672
2017-01-03 07:04:00+00:00,61.021250,61.052835,63.886790,64.039440,0.081265,0.00672


In [ ]:
data.to_pickle(PATH / "two_assets.pkl")

In [ ]:
data = df[["Bid_x", "Ask_x", "rub_rate", "usd_rate"]]
data.rename(columns={"Bid_x": "bid", "Ask_x": "ask"}, inplace=True)
data.head()

<ipython-input-28-27be0ee3bfcf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={"Bid_x": "bid", "Ask_x": "ask"}, inplace=True)


,bid,ask,rub_rate,usd_rate
timestamp,,,,
2017-01-03 07:00:00+00:00,60.973760,61.016500,0.081265,0.00672
2017-01-03 07:01:00+00:00,61.010855,61.068045,0.081265,0.00672
2017-01-03 07:02:00+00:00,61.037220,61.069850,0.081265,0.00672
2017-01-03 07:03:00+00:00,61.011955,61.048690,0.081265,0.00672
2017-01-03 07:04:00+00:00,61.021250,61.052835,0.081265,0.00672


In [ ]:
data.to_pickle(PATH / "2024.pkl")